In [24]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from answer_judge.vqaEval import VQAEval
from utils.seed import fix_seed

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
fix_seed(0)
data_reader=ReadData('/root/autodl-tmp/hallscope/VQAIDK/train_qwen')
data=data_reader.read_all()
random.shuffle(data)

In [26]:
df=pd.DataFrame(data)

In [27]:
df.head()

,img_path,question,answer,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,"[Given the question 'What is the animal?', is ...",[answerable],{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,"[Given the question 'See a bird anywhere?', is...",[answerable],{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[Given the question 'What shape is this room?'...,[answerable],{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Given the question 'What is the word at the b...,[unanswerable],{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Given the question 'Is the man wearing a hat?...,[answerable],{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None


In [28]:
df.shape

(5000, 5)

In [29]:
i=300

In [30]:
df.iloc[i]['img_path'],df.iloc[i]['most_likely']['response'],df.iloc[i]['question'],df.iloc[i]['answer']

(['/root/autodl-fs/coco_images/train/COCO_train2014_000000371376.jpg'],
 ['Unanswerable'],
 ["Given the question 'What OS does this computer have?', is the question answerable or unanswerable based on the image?\nPlease reply with 'Unanswerable' or 'Answerable'. \n"],
 ['unanswerable'])

In [31]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [32]:
embeddings=np.concatenate(embeddings,axis=0)

In [33]:
embeddings.shape

(5000, 29, 3584)

In [34]:
df['answer']=df['answer'].apply(lambda x: x[0])

In [35]:
gt_label=df['answer'].to_list()
gt_label

['answerable',
 'answerable',
 'answerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'answerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'answerable',
 'answerable',
 'answerable',
 'answerable',


In [36]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [37]:
rsps[0]

'Answerable'

In [38]:
is_correct=[]
for i,x in enumerate(gt_label):
    if rsps[i].lower()==x.lower():
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [39]:
is_correct.shape

(5000,)

In [40]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 5000 responses. 3098 answer correctly and 1902 answer wrongly


In [41]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:- validset_len]  # trainset index
val_idxs = train_val_idxs[- validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:3000
validset length:1000
testset length:1000
trainset postive: 1887 negative: 1113
valset postive: 594 negative: 406
testset postive: 617 negative: 383


In [42]:
all_embeddings = embeddings
all_embeddings.shape

(5000, 29, 3584)

In [43]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [44]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score
from ml_utils.metrics import auc_pr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [45]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCALinear(embedding_train[:,i_layer,:],n_components)
    scores=discriminator.get_score(X[:,i_layer,:])
    
    return roc_auc_score(y,scores)
    #split=discriminator.get_best_split(scores,y)
    #y_pred=scores>split
    #return accuracy_score(y,y_pred)
    #return balanced_accuracy_score(y,y_pred)
    #
    #split=discriminator.get_best_split(y)
    #y_preds=(scores>split)
    #return accuracy_score(y,y_preds)

In [52]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,11),
    'layer':range(0,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras,all_results=grid_search.search(return_results=True)

Grid Searching for best n_components,layer


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.4249037999037999
found best n_components:1,layer:1 Score: 0.5937494816805162
found best n_components:1,layer:2 Score: 0.5971164850475196
found best n_components:1,layer:7 Score: 0.6287173873380769


  0%|          | 0/29 [00:00<?, ?it/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

In [53]:
np.save('./VQAIDK_layers.npy',all_results,allow_pickle=True)

In [48]:
best_paras

{'n_components': 2, 'layer': 26, 'best_score': np.float64(0.7252699407871822)}

In [38]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [45]:
discriminator=PCALinear(embedding_train[:,best_layer,:],best_paras['n_components'])
scores=discriminator.get_score(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)
#best_split=scores[1000]

In [46]:
discriminator=PCALinear(embedding_train[:,best_layer,:],best_paras['n_components'])
scores=discriminator.get_score(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.6825 AUROC:0.7288723709288578


In [47]:
scores=discriminator.get_score(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.6825 AUROC:0.7172505658649526


Train Linear Prob to Get Better Results

In [48]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [49]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [50]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores_test)}  b-acc:{balanced_accuracy_score(y_test,pred_test)} auc-pr:{auc_pr(y_test,scores_test)}')

test set acc: 0.7255 f1:0.7895745496358758 auroc:0.7831268574193102  b-acc:0.6940225654848297 auc-pr:0.8704925434251023
